In [ ]:
import pandas as pd
import glob
import os

# 폴더 내의 .gz.parquet 파일을 검색합니다.
files = glob.glob('*.gz.parquet')
files

In [3]:
import pandas as pd
import glob
import os
import gc  
from tqdm import tqdm

def get_file_size(file):
    return os.path.getsize(file)

# 폴더 리스트
folders = ['company', 'estimates', 'exchangerate', 'financials', 'keydev', 
           'market', 'ownership', 'professional', 'transcript']

# 최대 파일 크기 설정 (예: 1GB)
MAX_FILE_SIZE = 1 * 1024 * 1024 * 1024  # 이 값을 조정하여 최대 파일 크기를 설정할 수 있습니다.

# 파일 리스트와 그 크기 가져오기
files_with_size = []
for folder in folders:
    file = glob.glob(f'{folder}/*.gz.parquet')[0]
    size = get_file_size(file)
    files_with_size.append((folder, file, size))

# 파일 크기 순으로 정렬
files_with_size.sort(key=lambda x: x[2])

# 각 파일을 처리
for folder, file, size in tqdm(files_with_size, desc="Processing files"):
    # 파일 크기가 최대 크기보다 크면 건너뛰기
    if size > MAX_FILE_SIZE:
        print(f"The file {file} is too large ({size} bytes). Skipping.")
        continue

    # Parquet 파일명
    parquet_filename = f"{folder}.parquet"

    # 이미 해당 이름의 Parquet 파일이 존재하는 경우 건너뛰기
    if os.path.exists(parquet_filename):
        print(f"{parquet_filename} already exists. Skipping.")
        continue
    
    # .gz.parquet 파일을 데이터프레임으로 읽기
    df = pd.read_parquet(file)
    print(f"{folder} 파일 읽기 완료")
    
    
    # 데이터프레임을 Parquet 파일로 저장
    df.to_parquet(parquet_filename)
    print(f"Saved {parquet_filename}")
    
    # 메모리 해제
    del df
    
    # 가비지 컬렉션 실행
    gc.collect()


Processing files:   0%|          | 0/9 [00:00<?, ?it/s]

다 읽었다


Processing files:  11%|█         | 1/9 [00:00<00:06,  1.15it/s]

Saved exchangerate.parquet
다 읽었다


Processing files:  22%|██▏       | 2/9 [00:03<00:14,  2.07s/it]

...

Saved professional.parquet
다 읽었다
Saved keydev.parquet


Processing files:  56%|█████▌    | 5/9 [00:36<00:39,  9.76s/it]

다 읽었다
Saved company.parquet


Processing files: 100%|██████████| 9/9 [02:14<00:00, 14.93s/it]

The file market\part-00000-tid-2679257485173306022-ebf58120-261f-486c-92af-e0b6cf6a33c5-14782-1-c000.gz.parquet is too large (1341743847 bytes). Skipping.
The file financials\part-00000-tid-3926431628187713411-a790f6d3-fba9-4590-86bb-309ca16e8e12-539-1-c000.gz.parquet is too large (2282087173 bytes). Skipping.
The file transcript\part-00000-tid-8297069538010164701-6b6a55d9-6263-40bd-811e-5da1c1553e6d-378-1-c000.gz.parquet is too large (2733575788 bytes). Skipping.


In [1]:
import pandas as pd
import numpy as np
import pyarrow
import gc
import time
import glob
import sys
import humanize
import math
import psutil
import gc
import simplejson
import skimage
import skimage.measure
from timeit import timeit
from time import sleep
from pyarrow.parquet import ParquetFile
import pyarrow
import pyarrow.parquet as pq
import signal
from contextlib import contextmanager

pd.set_option('display.max_columns',   500)
pd.set_option('display.max_colwidth',  None)

In [6]:
import os
import dask.dataframe as dd
import time
import psutil
import gc
from tqdm import tqdm

# 현재 메모리 사용량을 반환하는 함수
def get_memory_usage():
    return psutil.Process().memory_info().rss / 1024**2  # MB 단위

# 파일 이름에서 확장자를 제외한 이름 추출
file_path = "transcript.parquet"
base_name = os.path.basename(file_path)
folder_name, _ = os.path.splitext(base_name)

# 해당 이름으로 폴더 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Dask로 데이터의 열 정보만 불러오기
df_meta = dd.read_parquet(file_path, engine='fastparquet')

overall_start_time = time.time()  # 전체 작업 시작 시간 저장

# 각 열에 대해 고유한 값들을 계산하고 파일로 저장 (tqdm으로 진행률 표시)
for column in tqdm(df_meta.columns, desc="Processing columns"):
    start_time = time.time()  # 해당 열 처리 시작 시간 저장
    
    # 해당 열만 불러오기
    df_column = dd.read_parquet(file_path, columns=[column], engine='fastparquet')
    
    # 해당 열의 고유한 값들을 계산
    unique_values = df_column[column].unique().compute()
    
    # 고유한 값들을 폴더 내 파일로 저장
    filename = os.path.join(folder_name, f"unique_{folder_name}_{column}.csv")
    unique_values.to_csv(filename, index=False)
    
    elapsed_time = time.time() - start_time  # 해당 열 처리 소요 시간 계산
    memory_usage = get_memory_usage()  # 현재 메모리 사용량
    
    print(f"Completed column: {column}")
    print(f"Memory usage: {memory_usage:.2f} MB")
    print(f"Time taken: {elapsed_time:.2f} seconds")
    print("======================================")
    
    # 메모리 정리
    del df_column, unique_values
    gc.collect()  # 가비지 콜렉션 수행

overall_elapsed_time = time.time() - overall_start_time  # 전체 작업 소요 시간 계산
print(f"Total time taken for all columns: {overall_elapsed_time:.2f} seconds")

Processing columns:   0%|          | 0/24 [00:00<?, ?it/s]

Processing columns:   4%|▍         | 1/24 [00:00<00:15,  1.52it/s]

Completed column: companyId
Memory usage: 256.19 MB
Time taken: 0.61 seconds


Processing columns:   8%|▊         | 2/24 [00:12<02:34,  7.04s/it]

Completed column: companyName
Memory usage: 258.80 MB
Time taken: 11.45 seconds


...

Completed column: divAmount
Memory usage: 262.43 MB
Time taken: 0.77 seconds


Processing columns:  96%|█████████▌| 23/24 [02:37<00:04,  4.26s/it]

Completed column: netAmount
Memory usage: 261.52 MB
Time taken: 0.72 seconds


Processing columns: 100%|██████████| 24/24 [02:38<00:00,  6.60s/it]

Completed column: grossAmount
Memory usage: 262.33 MB
Time taken: 0.77 seconds
Total time taken for all columns: 158.43 seconds


# Groupby

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 31.15 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:5335,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 31.15 GiB
Comm: tcp://127.0.0.1:5390,Total threads: 4
Dashboard: http://127.0.0.1:5393/status,Memory: 3.89 GiB
Nanny: tcp://127.0.0.1:5338,


In [8]:
import dask.dataframe as dd
import pandas as pd
file_path = "transcript.parquet"
df = dd.read_parquet(file_path, engine='fastparquet')
df = df.persist()

In [9]:
df

,companyId,companyName,tickerSymbol,exchangeSymbol,ISOCode,pricingDate,exDate,payDate,recordDate,announcedDate,priceClose,volume,adjustmentFactor,VWAP,divAdjFactor,marketCap,TEV,sharesOutstanding,divFreqTypeName,supplementalTypeName,dividendTypeName,divAmount,netAmount,grossAmount
npartitions=2,,,,,,,,,,,,,,,,,,,,,,,,
,float64,string,string,string,string,datetime64[ns],datetime64[ns],datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,float64,float64,float64,float64,string,string,string,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
import os
import pandas as pd
import dask.dataframe as dd
import time
import gc
from tqdm import tqdm

def save_group_to_file(group, name, save_folder, groupby_column):
    """
    주어진 그룹 데이터를 파일로 저장하는 함수
    """
    save_path = os.path.join(save_folder, f"{groupby_column}_{name}.parquet")
    group.to_parquet(save_path, engine='fastparquet')
    del group
    gc.collect()

# 파일 경로 및 이름 설정
file_path = "transcript.parquet"
base_name = os.path.basename(file_path)
folder_name, _ = os.path.splitext(base_name)

# CSV 파일로부터 그룹화할 열 이름 추출
ticker_csv = 'transcript/unique_transcript_tickerSymbol.csv'
groupby_column = ticker_csv.split('_')[-1].replace('.csv', '')
group_names = pd.read_csv(ticker_csv).values.flatten()

# 저장할 폴더 준비
save_folder = os.path.join(folder_name, "grouped_by_" + groupby_column)
os.makedirs(save_folder, exist_ok=True)  # 폴더가 없으면 생성

# Dask로 데이터 불러오기
df_meta = dd.read_parquet(file_path, engine='fastparquet')

# Repartition if necessary
df_meta = df_meta.repartition(npartitions=8)

# Persist the data into memory to avoid recomputation
df_meta = df_meta.persist()

# 시작 시간 기록
overall_start_time = time.time()

# Dask의 groupby를 사용하여 그룹화 및 파일 저장

grouped = df_meta.groupby(groupby_column)

empty_meta = pd.DataFrame({'index': pd.Index([], dtype='object')}).set_index('index')
result = (grouped.apply(lambda group: save_group_to_file(group, group.name, save_folder, groupby_column),
                        meta=empty_meta)
          .compute())  # or use your Dask distributed client here

# 전체 작업 소요 시간 출력
print(f"Grouped data saved in folder: {save_folder}")
overall_elapsed_time = time.time() - overall_start_time
print(f"Total time taken for all tasks: {overall_elapsed_time:.2f} seconds")